<a href="https://colab.research.google.com/github/micah-shull/pipelines/blob/main/pipelines_16_ensemble_02_stacking_07_additional_modelsipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Additional Models

Adding more models to the VotingClassifier can help improve precision for class 1 and recall for class 0 by leveraging the strengths of different models. Here’s how you can add more models to your VotingClassifier:

### Steps:
1. **Identify Additional Models**: Select additional models that might complement the existing ones.
2. **Train the New Models**: Train these models and include them in the VotingClassifier.
3. **Evaluate the Combined Model**: Evaluate the performance of the VotingClassifier with the additional models.

### Example Additional Models:
- **XGBoost**: Another gradient boosting model that can help with precision and recall.
- **Support Vector Machine (SVM)**: Known for classification tasks, particularly useful with appropriate kernel settings.
- **Neural Network (MLPClassifier)**: Can capture complex patterns in the data.



### How the Voting Classifier Weights Models

The `VotingClassifier` in scikit-learn combines multiple models (classifiers) and predicts the class label based on the majority vote or the average of predicted probabilities. Here’s how it works:

- **Hard Voting**: Each classifier votes for a class, and the class with the most votes wins. If there’s a tie, one of the tied classes is randomly selected.
- **Soft Voting**: Each classifier predicts the probabilities of each class. These probabilities are averaged, and the class with the highest average probability is selected as the final prediction.

### Weights in Voting Classifier

The `VotingClassifier` can assign different weights to each model. These weights influence the contribution of each model’s prediction to the final prediction in the case of soft voting. The syntax for specifying weights is:

```python
voting_clf = VotingClassifier(estimators=[('model1', clf1), ('model2', clf2)], voting='soft', weights=[w1, w2])
```

In this syntax:
- `estimators` is a list of tuples, where each tuple consists of a model name and the model itself.
- `weights` is a list of weights corresponding to each model. The length of the weights list must match the number of models.

### Impact of Model Duplication

If you have a model that achieves high precision for class 1 and you add multiple copies of this model to the VotingClassifier, the following can occur:

1. **Increased Influence**: The duplicated model(s) will have a greater influence on the final prediction. If this model consistently predicts class 1 with high precision, it can increase the precision of class 1 in the VotingClassifier.
2. **Potential Overfitting**: Adding multiple copies of the same model can lead to overfitting. The ensemble may become less robust and more sensitive to the duplicated model’s predictions.
3. **Bias Towards Duplicated Model**: The VotingClassifier may become biased towards the predictions of the duplicated model, potentially ignoring the diverse predictions from other models.

### Example of Adding Weights to VotingClassifier

Here’s how you can add weights to the VotingClassifier:

```python
# Example models
clf1 = LogisticRegression(random_state=42)
clf2 = RandomForestClassifier(random_state=42)
clf3 = LGBMClassifier(random_state=42)

# VotingClassifier with weights
voting_clf = VotingClassifier(estimators=[
    ('lr', clf1),
    ('rf', clf2),
    ('lgbm', clf3),
    ('lgbm2', clf3),  # Duplicate model for higher weight
    ('lgbm3', clf3),  # Duplicate model for higher weight
], voting='soft', weights=[1, 1, 1, 1, 1])

# Fit and predict
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))
```

In this example, `LGBMClassifier` has been added three times, giving it a higher weight in the VotingClassifier.

### Summary

- **Soft Voting**: Probabilities are averaged, and the class with the highest average probability is selected.
- **Weights**: You can assign different weights to each model to control their influence on the final prediction.
- **Duplication**: Adding multiple copies of the same model can increase its influence but may also lead to overfitting.

By appropriately weighting the models or adding more instances of a high-performing model, you can potentially improve specific performance metrics like precision for class 1.

### Hard Voting Classifier

When using 'hard' voting in the VotingClassifier, the method predict_proba is not available because 'hard' voting only makes use of the class predictions from each model rather than their probabilities. Consequently, you cannot use the predict_with_class_specific_thresholds function that relies on predicted probabilities.

Instead, you need to directly use the predict method of the VotingClassifier when using 'hard' voting.

In [1]:
import sklearn
print(sklearn.__version__)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import joblib
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, classification_report
from loan_data_utils import load_and_preprocess_data

# Load the preprocessed data (assuming the function is defined in loan_data_utils)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
categorical_columns = ['sex', 'education', 'marriage']
target = 'default_payment_next_month'
X, y = load_and_preprocess_data(url, categorical_columns, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define the column transformer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])

# Load the previous best models and parameters
best_models = joblib.load('best_models.pkl')
with open('best_params.json', 'r') as json_file:
    best_params = json.load(json_file)

# Define additional models without class weights
additional_models = {
    'XGB': XGBClassifier(random_state=42),
    'MLP': MLPClassifier(random_state=42, max_iter=1000),
    'SVM': SVC(probability=True, random_state=42)
}

# Create pipelines for each additional model
additional_pipelines = {name: Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
                        for name, model in additional_models.items()}

# Add the additional models to the best models
best_models.update(additional_pipelines)

# Save the updated models and parameters
joblib.dump(best_models, 'best_models_extended.pkl')
with open('best_params_extended.json', 'w') as json_file:
    json.dump(best_params, json_file, indent=4)

# Create a list of (name, model) tuples for the VotingClassifier
estimators = [
    ('recall_class_1', best_models['recall_class_1']),
    ('precision_class_1', best_models['precision_class_1']),
    ('recall_class_0', best_models['recall_class_0']),
    ('precision_class_0', best_models['precision_class_0']),
    ('XGB', best_models['XGB']),
    ('MLP', best_models['MLP']),
    ('SVM', best_models['SVM'])
]

# Initialize the VotingClassifier with the best models
voting_clf = VotingClassifier(estimators=estimators, voting='soft')

# Fit the VotingClassifier on the training data
voting_clf.fit(X_train, y_train)

# Save the updated VotingClassifier model
joblib.dump(voting_clf, 'voting_classifier_extended.pkl')

print("Updated VotingClassifier model saved to 'voting_classifier_extended.pkl'")

# Load the optimal thresholds from the file
with open('optimal_thresholds.json', 'r') as json_file:
    optimal_thresholds = json.load(json_file)

# Function to apply class-specific thresholds
def predict_with_class_specific_thresholds(model, X, threshold_class_1, threshold_class_0):
    y_proba = model.predict_proba(X)
    y_pred = np.zeros(y_proba.shape[0])

    # Apply thresholds to obtain predictions
    y_pred[(y_proba[:, 1] >= threshold_class_1)] = 1  # Predict class 1 for probabilities above threshold_class_1
    y_pred[(y_proba[:, 0] >= threshold_class_0)] = 0  # Predict class 0 for probabilities above threshold_class_0

    return y_pred

# Predict with the updated VotingClassifier model using the optimal thresholds
threshold_class_1 = optimal_thresholds['threshold_class_1']
threshold_class_0 = optimal_thresholds['threshold_class_0']

y_pred_voting = predict_with_class_specific_thresholds(voting_clf, X_test, threshold_class_1, threshold_class_0)

# Evaluate the performance of the VotingClassifier
recall_1 = recall_score(y_test, y_pred_voting, pos_label=1)
precision_1 = precision_score(y_test, y_pred_voting, pos_label=1, zero_division=0)
recall_0 = recall_score(y_test, y_pred_voting, pos_label=0)
precision_0 = precision_score(y_test, y_pred_voting, pos_label=0, zero_division=0)
f1_macro = f1_score(y_test, y_pred_voting, average='macro')
accuracy = accuracy_score(y_test, y_pred_voting)

# Print the evaluation metrics
print(f'Recall Class 1: {recall_1:.4f}')
print(f'Precision Class 1: {precision_1:.4f}')
print(f'Recall Class 0: {recall_0:.4f}')
print(f'Precision Class 0: {precision_0:.4f}')
print(f'F1 Macro: {f1_macro:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_voting))


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007270 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001954 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightG

In [3]:
import joblib
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, classification_report
from loan_data_utils import load_and_preprocess_data

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the preprocessed data (assuming the function is defined in loan_data_utils)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
categorical_columns = ['sex', 'education', 'marriage']
target = 'default_payment_next_month'
X, y = load_and_preprocess_data(url, categorical_columns, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define the column transformer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])

# Load the previous best models and parameters
best_models = joblib.load('best_models.pkl')
with open('best_params.json', 'r') as json_file:
    best_params = json.load(json_file)

# Create a list of (name, model) tuples for the VotingClassifier
estimators = [
    ('recall_class_1', best_models['recall_class_1']),
    ('precision_class_1', best_models['precision_class_1']),
    ('recall_class_0', best_models['recall_class_0']),
    ('precision_class_0', best_models['precision_class_0'])
]

# Initialize the VotingClassifier with the best models using 'hard' voting
voting_clf = VotingClassifier(estimators=estimators, voting='hard')

# Fit the VotingClassifier on the training data
voting_clf.fit(X_train, y_train)

# Save the updated VotingClassifier model
joblib.dump(voting_clf, 'voting_classifier_hard.pkl')

print("Updated VotingClassifier model saved to 'voting_classifier_hard.pkl'")

# Predict with the updated VotingClassifier model
y_pred_voting = voting_clf.predict(X_test)

# Evaluate the performance of the VotingClassifier
recall_1 = recall_score(y_test, y_pred_voting, pos_label=1)
precision_1 = precision_score(y_test, y_pred_voting, pos_label=1, zero_division=0)
recall_0 = recall_score(y_test, y_pred_voting, pos_label=0)
precision_0 = precision_score(y_test, y_pred_voting, pos_label=0, zero_division=0)
f1_macro = f1_score(y_test, y_pred_voting, average='macro')
accuracy = accuracy_score(y_test, y_pred_voting)

# Print the evaluation metrics
print(f'Recall Class 1: {recall_1:.4f}')
print(f'Precision Class 1: {precision_1:.4f}')
print(f'Recall Class 0: {recall_0:.4f}')
print(f'Precision Class 0: {precision_0:.4f}')
print(f'F1 Macro: {f1_macro:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_voting))


[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002212 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002039 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [bin

In [4]:
import joblib
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, classification_report
from loan_data_utils import load_and_preprocess_data

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the preprocessed data (assuming the function is defined in loan_data_utils)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
categorical_columns = ['sex', 'education', 'marriage']
target = 'default_payment_next_month'
X, y = load_and_preprocess_data(url, categorical_columns, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define the column transformer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])

# Load the previous best models and parameters
best_models = joblib.load('best_models.pkl')
with open('best_params.json', 'r') as json_file:
    best_params = json.load(json_file)

# Define the models to be duplicated
recall_class_1_model = best_models['recall_class_1']
precision_class_1_model = best_models['precision_class_1']
recall_class_0_model = best_models['recall_class_0']

# Create a list of (name, model) tuples for the VotingClassifier
estimators = [
    ('recall_class_1_1', recall_class_1_model),
    ('recall_class_1_2', recall_class_1_model),
    ('recall_class_1_3', recall_class_1_model),
    ('precision_class_1_1', precision_class_1_model),
    ('precision_class_1_2', precision_class_1_model),
    ('precision_class_1_3', precision_class_1_model),
    ('recall_class_0_1', recall_class_0_model),
    ('recall_class_0_2', recall_class_0_model)
]

# Initialize the VotingClassifier with the duplicated models using 'soft' voting
voting_clf = VotingClassifier(estimators=estimators, voting='soft')

# Fit the VotingClassifier on the training data
voting_clf.fit(X_train, y_train)

# Save the updated VotingClassifier model
joblib.dump(voting_clf, 'voting_classifier_multiplied.pkl')

print("Updated VotingClassifier model saved to 'voting_classifier_multiplied.pkl'")

# Load the optimal thresholds from the file
with open('optimal_thresholds.json', 'r') as json_file:
    optimal_thresholds = json.load(json_file)

# Function to apply class-specific thresholds
def predict_with_class_specific_thresholds(model, X, threshold_class_1, threshold_class_0):
    y_proba = model.predict_proba(X)
    y_pred = np.zeros(y_proba.shape[0])

    # Apply thresholds to obtain predictions
    y_pred[(y_proba[:, 1] >= threshold_class_1)] = 1  # Predict class 1 for probabilities above threshold_class_1
    y_pred[(y_proba[:, 0] >= threshold_class_0)] = 0  # Predict class 0 for probabilities above threshold_class_0

    return y_pred

# Predict with the updated VotingClassifier model using the optimal thresholds
threshold_class_1 = optimal_thresholds['threshold_class_1']
threshold_class_0 = optimal_thresholds['threshold_class_0']

y_pred_voting = predict_with_class_specific_thresholds(voting_clf, X_test, threshold_class_1, threshold_class_0)

# Evaluate the performance of the VotingClassifier
recall_1 = recall_score(y_test, y_pred_voting, pos_label=1)
precision_1 = precision_score(y_test, y_pred_voting, pos_label=1, zero_division=0)
recall_0 = recall_score(y_test, y_pred_voting, pos_label=0)
precision_0 = precision_score(y_test, y_pred_voting, pos_label=0, zero_division=0)
f1_macro = f1_score(y_test, y_pred_voting, average='macro')
accuracy = accuracy_score(y_test, y_pred_voting)

# Print the evaluation metrics
print(f'Recall Class 1: {recall_1:.4f}')
print(f'Precision Class 1: {precision_1:.4f}')
print(f'Recall Class 0: {recall_0:.4f}')
print(f'Precision Class 0: {precision_0:.4f}')
print(f'F1 Macro: {f1_macro:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_voting))


[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.079441 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011635 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] 

#### Analysis of Results

The decrease in performance when duplicating the models suggests that the VotingClassifier might be overfitting to certain models, leading to a less robust ensemble.

### Observations:

1. **Decrease in Overall Metrics**:
   - Precision for Class 1 dropped from 0.68 to 0.43.
   - Recall for Class 1 dropped from 0.95 to 0.65.
   - Precision and recall for Class 0 also decreased, though not as drastically.

2. **Overfitting**:
   - The addition of multiple identical models can lead to overfitting, as these models dominate the voting process, reducing the diversity and robustness of the ensemble.

### Recommendations:

1. **Revert to the Best Performing Model**:
   - Given the performance drop, it’s advisable to revert to the best performing model without duplication.

2. **Optimize Voting Weights**:
   - Instead of duplicating models, consider optimizing the weights of each model in the VotingClassifier to give more influence to high-performing models.

3. **Resampling Techniques**:
   - Implement resampling techniques such as SMOTE, ADASYN, or undersampling to balance the class distribution, which might improve the recall and precision for class 1.

### Explanation:

1. **Optimized Weights**: Adjust the weights assigned to each model to give more influence to models that perform well for specific metrics.
2. **Train and Evaluate**: Train the updated VotingClassifier with the optimized weights and evaluate its performance.

By running this code, you can see if optimizing the weights of the VotingClassifier improves precision for class 1 and recall for class 0 while maintaining overall performance.

In [6]:
import joblib
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, classification_report
from loan_data_utils import load_and_preprocess_data

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the preprocessed data (assuming the function is defined in loan_data_utils)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
categorical_columns = ['sex', 'education', 'marriage']
target = 'default_payment_next_month'
X, y = load_and_preprocess_data(url, categorical_columns, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define the column transformer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])

# Load the previous best models and parameters
best_models = joblib.load('best_models.pkl')
with open('best_params.json', 'r') as json_file:
    best_params = json.load(json_file)

# Create a list of (name, model) tuples for the VotingClassifier
estimators = [
    ('recall_class_1', best_models['recall_class_1']),
    ('precision_class_1', best_models['precision_class_1']),
    ('recall_class_0', best_models['recall_class_0']),
    ('precision_class_0', best_models['precision_class_0'])
]

# Initialize the VotingClassifier with the best models using 'soft' voting and optimized weights
weights = [2, 1, 2, 1]  # Adjust these weights as needed
voting_clf = VotingClassifier(estimators=estimators, voting='soft', weights=weights)

# Fit the VotingClassifier on the training data
voting_clf.fit(X_train, y_train)

# Save the updated VotingClassifier model
joblib.dump(voting_clf, 'voting_classifier_weighted.pkl')

print("Updated VotingClassifier model saved to 'voting_classifier_weighted.pkl'")

# Load the optimal thresholds from the file
with open('optimal_thresholds.json', 'r') as json_file:
    optimal_thresholds = json.load(json_file)

# Function to apply class-specific thresholds
def predict_with_class_specific_thresholds(model, X, threshold_class_1, threshold_class_0):
    y_proba = model.predict_proba(X)
    y_pred = np.zeros(y_proba.shape[0])

    # Apply thresholds to obtain predictions
    y_pred[(y_proba[:, 1] >= threshold_class_1)] = 1  # Predict class 1 for probabilities above threshold_class_1
    y_pred[(y_proba[:, 0] >= threshold_class_0)] = 0  # Predict class 0 for probabilities above threshold_class_0

    return y_pred

# Predict with the updated VotingClassifier model using the optimal thresholds
threshold_class_1 = optimal_thresholds['threshold_class_1']
threshold_class_0 = optimal_thresholds['threshold_class_0']

y_pred_voting = predict_with_class_specific_thresholds(voting_clf, X_test, threshold_class_1, threshold_class_0)

# Evaluate the performance of the VotingClassifier
recall_1 = recall_score(y_test, y_pred_voting, pos_label=1)
precision_1 = precision_score(y_test, y_pred_voting, pos_label=1, zero_division=0)
recall_0 = recall_score(y_test, y_pred_voting, pos_label=0)
precision_0 = precision_score(y_test, y_pred_voting, pos_label=0, zero_division=0)
f1_macro = f1_score(y_test, y_pred_voting, average='macro')
accuracy = accuracy_score(y_test, y_pred_voting)

# Print the evaluation metrics
print(f'Recall Class 1: {recall_1:.4f}')
print(f'Precision Class 1: {precision_1:.4f}')
print(f'Recall Class 0: {recall_0:.4f}')
print(f'Precision Class 0: {precision_0:.4f}')
print(f'F1 Macro: {f1_macro:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_voting))


[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009221 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [bin

#### Comparison to Best Model:

#### Best Model's Performance:
- **Class 0**:
  - Precision: 0.99
  - Recall: 0.87
  - F1-Score: 0.93
- **Class 1**:
  - Precision: 0.68
  - Recall: 0.95
  - F1-Score: 0.80
- **Accuracy**: 0.89
- **Macro Average**:
  - Precision: 0.83
  - Recall: 0.91
  - F1-Score: 0.86
- **Weighted Average**:
  - Precision: 0.92
  - Recall: 0.89
  - F1-Score: 0.90

### Analysis:
1. **Decrease in Performance**:
   - The optimized weights did not perform as well as the best model.
   - Significant drops in precision, recall, and F1 scores for both classes.
   - Overall accuracy dropped from 0.89 to 0.73.

2. **Class 1 Precision**:
   - Precision for class 1 dropped from 0.68 to 0.43, indicating that a higher number of false positives were introduced.
   - Recall for class 1 also decreased from 0.95 to 0.64, showing that the model missed more actual class 1 instances.

3. **Class 0 Metrics**:
   - Precision for class 0 remained relatively high but dropped from 0.99 to 0.88.
   - Recall for class 0 decreased from 0.87 to 0.76, indicating the model missed more class 0 instances.

### Recommendations:
1. **Revert to Best Model**: Given the significant drop in performance, reverting to the best model configuration without additional weights is advisable.
2. **Resampling Techniques**: Implement resampling techniques such as SMOTE, ADASYN, or undersampling to balance the class distribution and potentially improve precision and recall for class 1.
3. **Feature Engineering**: Consider additional feature engineering to improve model performance.
4. **Parameter Tuning**: Further parameter tuning using cross-validation might help improve model performance.
5. **Ensemble Techniques**: Explore other ensemble techniques, such as stacking, to combine the strengths of different models more effectively.

By reverting to the best model and exploring these recommendations, you can aim to achieve better performance for both classes while maintaining overall accuracy.

The decrease in performance after modifying the VotingClassifier setup, including adding more models and adjusting weights, can be attributed to several factors:

#### Potential Reasons for Performance Decrease:

1. **Overfitting**:
   - **Duplicating Models**: When duplicating models or adding too many similar models, the ensemble can become overfitted to the specific patterns these models capture, reducing the diversity and robustness of the ensemble.
   - **Model Dominance**: By weighting certain models more heavily, the ensemble may become overly reliant on these models' predictions, leading to poorer generalization on unseen data.

2. **Model Interaction**:
   - **Conflict**: Different models may have conflicting predictions, and the process of averaging probabilities (soft voting) or majority voting (hard voting) might dilute the strengths of individual models.
   - **Thresholds**: The class-specific thresholds might not work as effectively with the new ensemble configuration, especially if the model probabilities are being averaged differently due to the new weights or added models.

3. **Model Quality**:
   - **Model Performance**: If the added models or the adjusted weights are not well-calibrated, they may introduce more noise than signal, leading to poorer overall performance.
   - **Class Imbalance**: Even with adjustments for class weights or thresholds, the inherent class imbalance might still pose a challenge, especially if the new models handle the imbalance differently than the original ones.

### Recommendations to Address Performance Decrease:

1. **Revert to Best Model Configuration**:
   - Use the best performing models and parameters identified previously without additional modifications or duplications.

2. **Ensure Proper Calibration**:
   - **Calibrate Models**: Ensure that the probabilities output by each model are well-calibrated. This can be done using techniques like Platt Scaling or Isotonic Regression.
   - **Cross-Validation**: Use cross-validation to tune the ensemble and model parameters more effectively.

3. **Resampling Techniques**:
   - **SMOTE/ADASYN**: Implement Synthetic Minority Over-sampling Technique (SMOTE) or Adaptive Synthetic Sampling (ADASYN) to create a more balanced training set.
   - **Undersampling**: Consider undersampling the majority class to achieve a better balance, though this may lead to a loss of information.

4. **Stacking Ensemble**:
   - Use a stacking ensemble where a meta-model learns to combine the predictions of the base models. This can often outperform simple voting ensembles by learning to trust different models in different scenarios.

5. **Feature Engineering**:
   - Create additional features that might help improve model performance, such as interaction terms or higher-order features.



In [7]:
import joblib
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, classification_report
from loan_data_utils import load_and_preprocess_data

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the preprocessed data (assuming the function is defined in loan_data_utils)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
categorical_columns = ['sex', 'education', 'marriage']
target = 'default_payment_next_month'
X, y = load_and_preprocess_data(url, categorical_columns, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define the column transformer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])

# Load the previous best models and parameters
best_models = joblib.load('best_models.pkl')
with open('best_params.json', 'r') as json_file:
    best_params = json.load(json_file)

# Create a list of (name, model) tuples for the VotingClassifier
estimators = [
    ('recall_class_1', best_models['recall_class_1']),
    ('precision_class_1', best_models['precision_class_1']),
    ('recall_class_0', best_models['recall_class_0']),
    ('precision_class_0', best_models['precision_class_0'])
]

# Initialize the VotingClassifier with the best models using 'soft' voting and optimized weights
weights = [2, 1, 2, 1]  # Adjust these weights as needed
voting_clf = VotingClassifier(estimators=estimators, voting='hard', weights=weights)

# Fit the VotingClassifier on the training data
voting_clf.fit(X_train, y_train)

# Save the updated VotingClassifier model
joblib.dump(voting_clf, 'voting_classifier_weighted.pkl')

print("Updated VotingClassifier model saved to 'voting_classifier_weighted.pkl'")

# Load the optimal thresholds from the file
with open('optimal_thresholds.json', 'r') as json_file:
    optimal_thresholds = json.load(json_file)

# Function to apply class-specific thresholds
def predict_with_class_specific_thresholds(model, X, threshold_class_1, threshold_class_0):
    y_proba = model.predict_proba(X)
    y_pred = np.zeros(y_proba.shape[0])

    # Apply thresholds to obtain predictions
    y_pred[(y_proba[:, 1] >= threshold_class_1)] = 1  # Predict class 1 for probabilities above threshold_class_1
    y_pred[(y_proba[:, 0] >= threshold_class_0)] = 0  # Predict class 0 for probabilities above threshold_class_0

    return y_pred

# Predict with the updated VotingClassifier model using the optimal thresholds
threshold_class_1 = optimal_thresholds['threshold_class_1']
threshold_class_0 = optimal_thresholds['threshold_class_0']

y_pred_voting = predict_with_class_specific_thresholds(voting_clf, X_test, threshold_class_1, threshold_class_0)

# Evaluate the performance of the VotingClassifier
recall_1 = recall_score(y_test, y_pred_voting, pos_label=1)
precision_1 = precision_score(y_test, y_pred_voting, pos_label=1, zero_division=0)
recall_0 = recall_score(y_test, y_pred_voting, pos_label=0)
precision_0 = precision_score(y_test, y_pred_voting, pos_label=0, zero_division=0)
f1_macro = f1_score(y_test, y_pred_voting, average='macro')
accuracy = accuracy_score(y_test, y_pred_voting)

# Print the evaluation metrics
print(f'Recall Class 1: {recall_1:.4f}')
print(f'Precision Class 1: {precision_1:.4f}')
print(f'Recall Class 0: {recall_0:.4f}')
print(f'Precision Class 0: {precision_0:.4f}')
print(f'F1 Macro: {f1_macro:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_voting))


[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011705 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003375 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightG

AttributeError: predict_proba is not available when voting='hard'

### Voting Classifier with Hard Voting

In [8]:
import joblib
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, classification_report
from loan_data_utils import load_and_preprocess_data

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the preprocessed data (assuming the function is defined in loan_data_utils)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
categorical_columns = ['sex', 'education', 'marriage']
target = 'default_payment_next_month'
X, y = load_and_preprocess_data(url, categorical_columns, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define the column transformer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])

# Load the previous best models and parameters
best_models = joblib.load('best_models.pkl')
with open('best_params.json', 'r') as json_file:
    best_params = json.load(json_file)

# Create a list of (name, model) tuples for the VotingClassifier
estimators = [
    ('recall_class_1', best_models['recall_class_1']),
    ('precision_class_1', best_models['precision_class_1']),
    ('recall_class_0', best_models['recall_class_0']),
    ('precision_class_0', best_models['precision_class_0'])
]

# Initialize the VotingClassifier with the best models using 'hard' voting
voting_clf = VotingClassifier(estimators=estimators, voting='hard')

# Fit the VotingClassifier on the training data
voting_clf.fit(X_train, y_train)

# Save the updated VotingClassifier model
joblib.dump(voting_clf, 'voting_classifier_hard_best.pkl')

print("Reverted to best VotingClassifier model with 'hard' voting saved to 'voting_classifier_hard_best.pkl'")

# Predict with the updated VotingClassifier model
y_pred_voting = voting_clf.predict(X_test)

# Evaluate the performance of the VotingClassifier
recall_1 = recall_score(y_test, y_pred_voting, pos_label=1)
precision_1 = precision_score(y_test, y_pred_voting, pos_label=1, zero_division=0)
recall_0 = recall_score(y_test, y_pred_voting, pos_label=0)
precision_0 = precision_score(y_test, y_pred_voting, pos_label=0, zero_division=0)
f1_macro = f1_score(y_test, y_pred_voting, average='macro')
accuracy = accuracy_score(y_test, y_pred_voting)

# Print the evaluation metrics
print(f'Recall Class 1: {recall_1:.4f}')
print(f'Precision Class 1: {precision_1:.4f}')
print(f'Recall Class 0: {recall_0:.4f}')
print(f'Precision Class 0: {precision_0:.4f}')
print(f'F1 Macro: {f1_macro:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_voting))


[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003522 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001919 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [bin

To incorporate optimal thresholds while including multiple models for each metric and effectively utilize class-specific thresholds, you can employ a more advanced approach than simple 'hard' voting. One effective method is to use a weighted average of probabilities with additional logic to apply class-specific thresholds.

### Steps to Achieve This:

1. **Soft Voting for Probability Calculation**: Use soft voting to get probabilities from multiple models.
2. **Custom Threshold Application**: Apply custom class-specific thresholds based on the predicted probabilities.
3. **Include Multiple Models for Each Metric**: Duplicate models to give more weight to those that perform well for specific metrics.

### Explanation:

1. **Soft Voting**: Use soft voting to calculate probabilities from multiple models.
2. **Custom Thresholds**: Apply class-specific thresholds to these probabilities to get the final predictions.
3. **Include Multiple Models**: Duplicate models to give more weight to those that perform well for specific metrics.

By running this code, you can evaluate the performance of the VotingClassifier with soft voting, class-specific thresholds, and multiple models for each metric. This approach combines the advantages of probability averaging with the benefits of class-specific thresholds and model weighting.

In [12]:
import joblib
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, classification_report
from loan_data_utils import load_and_preprocess_data

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load the preprocessed data (assuming the function is defined in loan_data_utils)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
categorical_columns = ['sex', 'education', 'marriage']
target = 'default_payment_next_month'
X, y = load_and_preprocess_data(url, categorical_columns, target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define the column transformer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(drop='first'))
        ]), categorical_features)
    ])

# Load the previous best models and parameters
best_models = joblib.load('best_models.pkl')
with open('best_params.json', 'r') as json_file:
    best_params = json.load(json_file)

# Define the models to be duplicated
recall_class_1_model = best_models['recall_class_1']
precision_class_1_model = best_models['precision_class_1']
recall_class_0_model = best_models['recall_class_0']
precision_class_0_model = best_models['precision_class_0']

# Create a list of (name, model) tuples for the VotingClassifier
estimators = [
    ('recall_class_1_1', recall_class_1_model),
    ('recall_class_1_2', recall_class_1_model),
    # ('recall_class_1_3', recall_class_1_model),
    ('precision_class_1_1', precision_class_1_model),
    ('precision_class_1_2', precision_class_1_model),
    # ('precision_class_1_3', precision_class_1_model),
    ('recall_class_0_1', recall_class_0_model),
    # ('recall_class_0_2', recall_class_0_model),
    ('precision_class_0_1', precision_class_0_model)
]

# Initialize the VotingClassifier with the duplicated models using 'soft' voting
voting_clf = VotingClassifier(estimators=estimators, voting='soft')

# Fit the VotingClassifier on the training data
voting_clf.fit(X_train, y_train)

# Save the updated VotingClassifier model
joblib.dump(voting_clf, 'voting_classifier_soft_multiplied.pkl')

print("Updated VotingClassifier model with soft voting and duplicated models saved to 'voting_classifier_soft_multiplied.pkl'")

# Load the optimal thresholds from the file
with open('optimal_thresholds.json', 'r') as json_file:
    optimal_thresholds = json.load(json_file)

# Function to apply class-specific thresholds
def predict_with_class_specific_thresholds(model, X, threshold_class_1, threshold_class_0):
    y_proba = model.predict_proba(X)
    y_pred = np.zeros(y_proba.shape[0])

    # Apply thresholds to obtain predictions
    y_pred[(y_proba[:, 1] >= threshold_class_1)] = 1  # Predict class 1 for probabilities above threshold_class_1
    y_pred[(y_proba[:, 0] >= threshold_class_0)] = 0  # Predict class 0 for probabilities above threshold_class_0

    return y_pred

# Predict with the updated VotingClassifier model using the optimal thresholds
threshold_class_1 = optimal_thresholds['threshold_class_1']
threshold_class_0 = optimal_thresholds['threshold_class_0']

y_pred_voting = predict_with_class_specific_thresholds(voting_clf, X_test, threshold_class_1, threshold_class_0)

# Evaluate the performance of the VotingClassifier
recall_1 = recall_score(y_test, y_pred_voting, pos_label=1)
precision_1 = precision_score(y_test, y_pred_voting, pos_label=1, zero_division=0)
recall_0 = recall_score(y_test, y_pred_voting, pos_label=0)
precision_0 = precision_score(y_test, y_pred_voting, pos_label=0, zero_division=0)
f1_macro = f1_score(y_test, y_pred_voting, average='macro')
accuracy = accuracy_score(y_test, y_pred_voting)

# Print the evaluation metrics
print(f'Recall Class 1: {recall_1:.4f}')
print(f'Precision Class 1: {precision_1:.4f}')
print(f'Recall Class 0: {recall_0:.4f}')
print(f'Precision Class 0: {precision_0:.4f}')
print(f'F1 Macro: {f1_macro:.4f}')
print(f'Accuracy: {accuracy:.4f}')

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_voting))


[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003554 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 5309, number of negative: 18691
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001995 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3276
[LightGBM] [Info] Number of data points in the train set: 24000, number of used features: 30
[LightGBM] [Info] [bin